In [30]:
import pandas as pd
import numpy as np
import sklearn
import random
import nltk
import string
import re
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [3]:
#I don't think you guys would need to run this at all
import os
os.getcwd()
os.chdir('d:\\Harris Course work\\MLPP\\ml_proj\\obamacare-ml-project')

In [16]:
#ChatGPT support
# set the file path and chunk size
file_path = 'data/yelp_academic_dataset_review.json'
chunk_size = 1000

# determine the total number of records in the file
with open(file_path, 'r', encoding='utf-8') as f:
    num_records = sum(1 for line in f)

# determine the indices of the records to load
subset_size = int(num_records * 0.1)  # load 10% of the records
indices = random.sample(range(num_records), subset_size)

# load the random subset of records in chunks
chunks = []
json_reader = pd.read_json(file_path, lines=True, chunksize=chunk_size)
for chunk in json_reader:
    filtered_chunk = chunk[chunk.index.isin(indices)]
    if not filtered_chunk.empty:
        chunks.append(filtered_chunk)

# concatenate the chunks into a single DataFrame
df = pd.concat(chunks)


In [21]:
df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
15,OAhBYw8IQ6wlfw1owXWRWw,1C2lxzUo1Hyye4RFIXly3g,BVndHaLihEYbr76Z0CMEGw,5,0,0,0,"Great place for breakfast! I had the waffle, w...",2014-10-11 16:22:06
20,vBK79c3_1Ff_oqkh5VpfGg,Ohhrhu1RkqfVciIVx_W5HQ,nRKndeZLQ3eDL10UMwS2rQ,5,2,5,2,HOLY SMOKES!\n\nactual pumpkin pie mixed in wi...,2009-10-13 19:49:51
21,ZVvhc3Go7v5I8XTiVoWmqQ,WBpQDAZymU0dhIqXcACGNw,ut6fi2W2YaipNOqvi7e0jw,3,0,0,0,"Upland is a brewery based out of Bloomington, ...",2014-11-12 14:12:20
25,qS6kE7CDoDagyPZwmueJaQ,zoBajEyVA0z4IjbFsMJksg,c-IgS6Pk6vMyax7Rbr38eA,4,0,0,0,Went for lunch. Beef brisket sandwich was awes...,2015-06-08 19:45:48


In [22]:
df.loc[:,'stars'].value_counts(normalize=True)

5    0.462215
4    0.207635
1    0.153413
3    0.099168
2    0.077569
Name: stars, dtype: float64

In [26]:
yelp_reviews = df.loc[:, ['text','stars', 'useful','funny','cool']]

In [31]:
yelp_reviews.shape

(699028, 5)

In [27]:
yelp_reviews.head()

,text,stars,useful,funny,cool
2,Family diner. Had the buffet. Eclectic assortm...,3,0,0,0
15,"Great place for breakfast! I had the waffle, w...",5,0,0,0
20,HOLY SMOKES!\n\nactual pumpkin pie mixed in wi...,5,2,5,2
21,"Upland is a brewery based out of Bloomington, ...",3,0,0,0
25,Went for lunch. Beef brisket sandwich was awes...,4,0,0,0


In [32]:
# Tokenizing and lemmatizing: I had some issue switching to Eshan's branch.
# Please edit if I missed a step
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\soumy\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\soumy\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\soumy\AppData\Roaming\nltk_data...


True

In [33]:
#Pre-processing strings
yelp_reviews.loc[:,'text'] = yelp_reviews.loc[:,'text'].str.lower()

In [34]:
#Removing punctuation
punc = r'[{}]+'.format(string.punctuation)
yelp_reviews.loc[:,'text'] = yelp_reviews.loc[:,'text'].apply(lambda x: re.sub(punc,'',x))

In [35]:
yelp_reviews.loc[:,'text'] = yelp_reviews.loc[:,'text'] .apply(lambda x: nltk.word_tokenize(x))

In [ ]:
stop_words = nltk.corpus.stopwords.words('english')
yelp_reviews.loc[:,'text'] = yelp_reviews.loc[:,'text'].apply(lambda x: [i for i in x if i not in stop_words])

In [ ]:
lemmatizer = nltk.stem.WordNetLemmatizer()
yelp_reviews.loc[:,'text'] = yelp_reviews.loc[:,'text'].apply(lambda x: [lemmatizer.lemmatize(i) for i in x])